# Build Gender Classificatio Prototype models

## Import all libraries

In [1]:
import streamlit as st
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
from PIL import Image, ImageOps
import cv2
import tensorflow as tf

#load the model
@st.cache
def load_model(model_path):
    return load_model(model_path)

def prepare_img(filepath):
    img_ = cv2.imread(filepath)
    resize = tf.image.resize(img_, (224,224))
    return np.expand_dims(resize/255, 0)

model = load_model("model_1.h5")
st.title("Hi! this is website that can detect gender from a given image")
image_streamlit  = st.file_uploader("Upload your image",type=["jpg","png","jpeg"])
if image_streamlit is None:
    pass
else :
    image_upload = Image.open(image_streamlit)
    image = prepare_img(image_upload)
    prediction = model.predict(image)
    st.write(prediction)

KeyboardInterrupt: 

In [2]:
import os
import datetime
import tensorflow as tf
import shutil
import cv2
import imghdr
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Setup the data path

In [ ]:
main_dir = "/kaggle/input/gender-classification-dataset/"
# Setup the train and test directories
project_name = "GenderClassification"
train_dir = f"{main_dir}Training/"
test_dir = f"{main_dir}Validation/"
labels = ["female","male"]

## Create all the fonctions we will need later

In [ ]:
# Create walk through directory fonction 
def walk_through_dir(main_dir,folder_name,labels):
    path_ = main_dir + folder_name +"/"+labels[0]
    path = main_dir + folder_name +"/"+labels[1]

    print(f"In the {folder_name} folder  with {labels[0]} class there is {len(os.listdir(path_))} images")
    print(f"In the {folder_name} folder  with {labels[1]} class there is {len(os.listdir(path))} images")    

In [ ]:
walk_through_dir(main_dir,"Validation",labels),walk_through_dir(main_dir,"Training",labels),

In [ ]:
def view_random_image(target_dir, target_class):
    target_folder = target_dir+target_class
    random_image = random.sample(os.listdir(target_folder), 1)
    img = mpimg.imread(target_folder + "/" + random_image[0])
    plt.imshow(img)
    plt.title(target_class)
    plt.axis("off");
    print(f"Image shape: {img.shape}") # show the shape of the image
    return img

def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
    print(f"Saving TensorBoard log files to: {log_dir}")
    return tensorboard_callback
def plot_loss_curves(history):
    
    """
    Returns separate loss curves for training and validation metrics.
    """ 
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']
    epochs = range(len(history.history['loss']))
  # Plot loss
    plt.plot(epochs, loss, label='training_loss')
    plt.plot(epochs, val_loss, label='val_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()

  # Plot accuracy
    plt.figure()
    plt.plot(epochs, accuracy, label='training_accuracy')
    plt.plot(epochs, val_accuracy, label='val_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend()

    
def prepare_img(filepath):
    img_ = cv2.imread(filepath)
    resize = tf.image.resize(img_, (224,224))
    return np.expand_dims(resize/255, 0)

def predict(model,filepath):
    image = prepare_img(filepath)
    image1 = mpimg.imread(filepath)
    y_pred = model.predict(image)
    plt.imshow(image1)
    print(y_pred)
    if y_pred > 0.50: 
        print(f'Predicted class is Male')
        plt.title("Male")
    else:
        plt.title("Female")
        print(f'Predicted class is Female')
    plt.show()

## Check our data that we are dealing with

In [ ]:
img = view_random_image(train_dir,random.choice(labels))

## load our data to the model

In [ ]:
train_data = tf.keras.utils.image_dataset_from_directory(train_dir,image_size=(224,224))
test_data = tf.keras.utils.image_dataset_from_directory(test_dir,image_size=(224,224))

In [ ]:
data_iterator = train_data.as_numpy_iterator()
batch = data_iterator.next()
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
train_data = train_data.map(lambda x,y:(x/255,y))
test_data = test_data.map(lambda x,y:(x/255,y))

## Build The models

### Model 1

#### Setup the layers

In [ ]:
model_1 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters=10, 
                         kernel_size=3, # can also be (3, 3)
                         activation="relu", 
                         input_shape=(224, 224, 3)), # first layer specifies input shape (height, width, colour channels)
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=2, # pool_size can also be (2, 2)
                            padding="valid"), # padding can also be 'same'
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.Conv2D(10, 3, activation="relu"), # activation='relu' == tf.keras.layers.Activations(tf.nn.relu)
  tf.keras.layers.MaxPool2D(2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1, activation="sigmoid") # binary activation output
])

model_1.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])
model_1.summary()

#### Fit the model

In [ ]:
history_1 = model_1.fit(train_data,
          epochs=15,
          callbacks=[create_tensorboard_callback(dir_name="logdir_tensorboard",experiment_name="model_1")],
          validation_data=test_data)

#### Evaluate the model performance

In [ ]:
plot_loss_curves(history_1)

In [ ]:
model_1.evaluate(test_data)

In [ ]:
random_class = random.choice(labels)
random_img = random.choice(os.listdir(test_dir+random_class))
img = test_dir+random_class+"/"+random_img
predict(model_1,img)

### Model 2

#### Setup the layers

In [ ]:
model_2 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters=10, 
                         kernel_size=3, 
                         activation="relu", 
                         input_shape=(224, 224, 3)), 
  tf.keras.layers.MaxPool2D(pool_size=2, 
                            padding="valid"),
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=2, 
                            padding="valid"), 
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
tf.keras.layers.MaxPool2D(pool_size=2, 
                            padding="valid"),
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1, activation="sigmoid")
])

model_2.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])
model_2.summary()

#### Fit the model

In [ ]:
history_2 = model_2.fit(train_data,
          epochs=15,
          callbacks=[create_tensorboard_callback(dir_name="logdir_tensorboard",experiment_name="model_2")],
          validation_data=test_data)

#### Evaluate the model performance

In [ ]:
plot_loss_curves(history_2)

In [ ]:
model_2.evaluate(test_data)

In [ ]:
random_class = random.choice(labels)
random_img = random.choice(os.listdir(test_dir+random_class))
img = test_dir+random_class+"/"+random_img
predict(model_2,img)

### Model 3

#### Setup the layers

In [ ]:
model_3 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters=100, 
                         kernel_size=3, 
                         activation="relu", 
                         input_shape=(224, 224, 3)), 
  tf.keras.layers.MaxPool2D(pool_size=2, 
                            padding="valid"),
  tf.keras.layers.Conv2D(65, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=2, 
                            padding="valid"), 
  tf.keras.layers.Conv2D(65, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=2, 
                            padding="valid"),
  tf.keras.layers.Conv2D(32, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=2, 
                            padding="valid"),
  tf.keras.layers.Conv2D(32, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=2, 
                            padding="valid"),
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=2, 
                            padding="valid"),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1, activation="sigmoid")
])

model_3.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])
model_3.summary()

#### Fit the model

In [ ]:
history_3 = model_3.fit(train_data,
          epochs=15,
          callbacks=[create_tensorboard_callback("logdir_tensorboard","model_3")],
          validation_data=test_data)

#### Evaluate the model performance

In [ ]:
plot_loss_curves(history_3)

In [ ]:
random_class = random.choice(labels)
random_img = random.choice(os.listdir(test_dir+random_class))
img = test_dir+random_class+"/"+random_img
predict(model_3,img)

## Keep Track all the models performance

In [ ]:
len(test_data.as_numpy_iterator().next()[:4])

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()
index = ["model_1","model_2","model_3"]
histories = [history_1,history_2,history_3]
list_val_acc = []
models_data = []
for model in models:
    X,y = (test_data.as_numpy_iterator().next()[0],test_data.as_numpy_iterator().next()[1])
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)
    models_data.append([pre.result().numpy(),re.result().numpy(),acc.result().numpy()])
for history in histories :
    list_val_acc.append(history.history['val_accuracy'][-1])

    
columns = ["Precision","Recall","BinaryAccuracy","ValAccuracy"]
df = pd.DataFrame(data=models_data,columns= columns[:-1])
df["ValAccuracy"] = list_val_acc
df.index = index
print(df)
for col in columns:
    print(f"{col} : {df[col].max()} with {df.loc[df[col] == df[col].max()].index[0]}")
df.to_csv("models_perfomance.csv")

In [ ]:
x_test = test_data.as_numpy_iterator().next()[0]
y_pred = model_1.predict(x_test)
y_pred = y_pred.argmax(axis=1)

In [ ]:
y_classes =  test_data.as_numpy_iterator().next()[1]
y_classes,y_pred

## Download the models on our local directory

In [ ]:
models = [model_1,model_2,model_3]
for i,model in enumerate(models):
    model.save(f'model_{i+1}.h5')

## Test the model on real Image from the web


In [ ]:
model_1 = load_model("model_1.h5")
model_1.evaluate(test_data)

In [ ]:
from google.colab.patches import cv2_imshow

# Load the models
model_1 = load_model("model_1.h5")

image_path = "<image url here>"

r = requests.get(image_path, allow_redirects=True)
# Save the image in our local dir
open('image.jpg', 'wb').write(r.content)
# Down the algo of face detection
alg = "https://raw.githubusercontent.com/kipr/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml"
r = requests.get(alg, allow_redirects=True)
open("haarcascade_frontalface_default.xml","wb").write(r.content)

#Setup the file path and the algo path
image = "image.jpg"
alg = "haarcascade_frontalface_default.xml"
img = cv2.imread(image)
haar_cascade = cv2.CascadeClassifier(alg)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = haar_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
print(faces)
# loop through all the faces in the image and get the crop face of each person then predict it take the crop face then put the predict label
for i,(x, y, w, h) in enumerate(faces):
    cv2.rectangle(img, (x, y - 20), (x + w, y), (0, 0, 255), -1)
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)
    face_crop = img[y:y+h, x:x+w]
    pict_name = f'cut_face{i}.jpg'
    cv2.imwrite(pict_name, face_crop)
    pred = model_1.predict(prepare_img(pict_name))
    print(pred)
    class_ = int(pred.round()[:,0])
    cv2.putText(img, f'{labels[class_]}', (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.7 ,(255,255,255), 1)
cv2_imshow(img)
cv2.waitKey()

## Test the model on webcame

In [ ]:
from tensorflow.keras.models import load_model
model = load_model("model_3.h5")
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

cap=cv2.VideoCapture(0)

labels_dict={1:'Male',0:'Female'}
color_dict={0:(0,0,255),1:(0,255,0)}

In [ ]:
while(True):
    ret,img=cap.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,3)  

    for (x,y,w,h) in faces:
        face_img=gray[y:y+w,x:x+w]
        pict_name = 'cut_face.jpg'
        cv2.imwrite(pict_name, face_img)
        pred = model.predict(prepare_img(pict_name))
        pred_pro = "%.2f" % pred[0][0]
        class_ = int(pred.round()[:,0])      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[class_],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[class_],-1)
        cv2.putText(img, f"{labels[class_]}:{pred_pro}%", (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    cv2.imshow('Result',img)
    k=cv2.waitKey(1)
    
    if k==ord("q"):
        break
        
cv2.destroyAllWindows()
cap.release()